In [ ]:
# Import librairies 
import dash
from dash import dcc, html, Output, Input, dash_table
import pandas as pd
import plotly.graph_objects as go
from utils.load_force_data import load_force_plate_data
from utils.plot_force_curve import plot_force_plate_curve
import os

# --- Initialisation de l'app ---
app = dash.Dash(__name__, title="Analyse CMJ – Plateforme de force")

# --- Chemin des données ---
default_path = "/Users/Armand/Desktop/Python/VALD" # -> à changer plus tard

# --- Layout ---
app.layout = html.Div([
    html.H2("Analyse CMJ – Plateforme de force"),

    html.Div([
        html.Label("Chemin des données :"),
        dcc.Input(id="data_path", type="text", value=default_path, style={"width": "80%"}),
        html.Button("Charger les fichiers", id="load_btn", n_clicks=0)
    ], style={"marginBottom": "20px"}),

    html.Div(id="file_selector"),

    dcc.Graph(id="force_plot", style={"height": "60vh"}),

    html.Div(id="summary_table")
])


# --- Callback : charger les fichiers ---
@app.callback(
    Output("file_selector", "children"),
    Input("load_btn", "n_clicks"),
    Input("data_path", "value")
)
def list_files(n, path):
    if not os.path.exists(path):
        return html.Div("⚠️ Le chemin spécifié n'existe pas.")
    files = [f for f in os.listdir(path) if "Nolan TRAORE" in f and f.endswith(".csv")]
    if not files:
        return html.Div("Aucun fichier trouvé pour Nolan TRAORE.")
    return html.Div([
        html.Label("Sélectionner un fichier :"),
        dcc.Dropdown(
            id="file_selected",
            options=[{"label": f, "value": os.path.join(path, f)} for f in files],
            value=os.path.join(path, files[0]),
            style={"width": "80%"}
        )
    ])


# --- Callback : afficher la courbe ---
@app.callback(
    Output("force_plot", "figure"),
    Output("summary_table", "children"),
    Input("file_selector", "children"),
    Input("data_path", "value"),
    Input("load_btn", "n_clicks"),
    Input("file_selected", "value"),
    prevent_initial_call=True
)
def update_plot(_, path, n, file_path):
    if not file_path:
        return go.Figure(), html.Div()

    df = load_force_plate_data(file_path)
    fig = plot_force_plate_curve(df)

    summary = pd.DataFrame({
        "N_points": [len(df)],
        "Fréquence (Hz)": [1000],
        "Force min (N)": [df["Force"].min()],
        "Force max (N)": [df["Force"].max()]
    })

    table = dash_table.DataTable(
        columns=[{"name": i, "id": i} for i in summary.columns],
        data=summary.to_dict("records"),
        style_table={"width": "60%"}
    )
    return fig, table


if __name__ == "__main__":
    app.run_server(debug=True)
